# Tensor Pipeline

## Import Libraries

In [1]:
import sys
sys.path.append('../Code Files')
sys.path.append('../Data')

In [2]:
import datetime
import numpy as np
import dill
from KFoldCV_SGD import KFoldCV
from train_test_SGD import train_test
from DataGeneration_SGD import *

## Import Data

In [3]:
import pickle
pkl_file = "../Data/Bounded_Var_Time_2024-03-31 20_19_12, intercept_5,n_train_5000, n_test_1000, tensor_dimensions_[32 32], tensor_mode_ranks_[4 4], separation_rank_2.pkl"
tensor_dimensions = np.array([32, 32])
tensor_mode_ranks = np.array([4, 4])
separation_rank = 2


file= open(pkl_file, 'rb')
data = pickle.load(file)
file.close()

X_train_Full = data[0]
print(data[0].shape)

Y_train_Full = data[1]
print(data[1].shape)

X_test_Full = data[2]
print(data[2].shape)

Y_test_Full = data[3]
print(data[3].shape)

B_tensored = data[4]
print(data[4].shape)

(5000, 32, 32)
(5000,)
(1000, 32, 32)
(1000,)
(32, 32)


## Subset Data

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

n_train_all = [5000]
n_test = 1000

shape_train = X_train_Full.shape
shape_test = X_test_Full.shape
print(f"Training Data Shape: {shape_train}, Test Data Shape: {shape_test}")

#Vectorize X_train_Full and X_test_Full!
X_train_Full = X_train_Full.reshape(X_train_Full.shape[0], -1)
X_test_Full = X_test_Full.reshape(X_test_Full.shape[0], -1)

#Scale according to MinMaxScaler
scaler = MinMaxScaler()
X_train_Full = scaler.fit_transform(X_train_Full)
X_test_Full = scaler.transform(X_test_Full)

#Reshape back to original shape
X_train_Full = X_train_Full.reshape(shape_train)
X_test_Full = X_test_Full.reshape(shape_test)

Training Data Shape: (5000, 32, 32), Test Data Shape: (1000, 32, 32)


In [5]:
for i,n_train in enumerate(n_train_all):
  print('Number of Samples:',n_train,'---------------------------------------------------------------------------------------------------------------------------')

  #Subset X_train and Y_train
  X_train = X_train_Full[0:(n_train),:,:]
  Y_train = Y_train_Full[0:(n_train)]

  #Subset X_test and Y_test
  X_test = X_test_Full[0:(n_test),:,:]
  Y_test = Y_test_Full[0:(n_test)]

  #For now, define finite alpha set that we are searching over
  alphas = [100]

  #Define Number of Folds we want
  k_folds = 10
  hypers = {'max_iter': 200, 'threshold': 1e-4, 'ranks': tuple(tensor_mode_ranks), 'separation_rank': separation_rank}
  lambda1, validation_normalized_estimation_error, validation_nmse_losses, validation_correlations, validation_R2_scores, objective_function_information = KFoldCV(X_train, Y_train, B_tensored, alphas, k_folds, hypers, intercept= False)

  hypers = {'max_iter': 200, 'threshold': 1e-4, 'ranks': tuple(tensor_mode_ranks), 'separation_rank': separation_rank}
  normalized_estimation_error, test_nmse_loss, test_R2_loss, test_correlation, objective_function_values = train_test(X_train, Y_train, X_test, Y_test, B_tensored, lambda1, hypers, intercept= False)

  #Get current time and store in variable
  formatted_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  max_iter = hypers['max_iter']
  pkl_file = f"../Data/ExecutionTime_intercept_0_{formatted_time}, n_train_{n_train},n_test_{n_test}, tensor_dimensions:{tensor_dimensions}, tensor_mode_= ranks:{tensor_mode_ranks}, separation_rank:{separation_rank}, max_iter={max_iter}.pkl"

  print("Error Record on Training _ After K-Fold CV")
  print("Validation Normalized Estimation Error: ", validation_normalized_estimation_error)
  print("Validation NMSE Losses: ", validation_nmse_losses)
  print("Validation Correlations: ", validation_correlations)
  print("Validation R2 Scores: ", validation_R2_scores)

  print("Error Report on Testing _ With best Lambda")
  print("Alpha chosen for model: ", lambda1)
  print("Test Normalized Estimation Error: ", normalized_estimation_error)
  print("Test NMSE Loss: ", test_nmse_loss)
  print("Test R2 Loss: ", test_R2_loss)
  print("Test Correlation: ", test_correlation)

  with open(pkl_file, "wb") as file:
    dill.dump((X_train_Full, Y_train_Full, X_test_Full, Y_test_Full, B_tensored, lambda1, validation_normalized_estimation_error, validation_nmse_losses, validation_correlations, validation_R2_scores, objective_function_information, normalized_estimation_error, test_nmse_loss, test_R2_loss, test_correlation, objective_function_values), file)

Number of Samples: 5000 ---------------------------------------------------------------------------------------------------------------------------
intercept is not initialized
intercept is not initialized
intercept is not initialized
intercept is not initialized
intercept is not initialized
intercept is not initialized
intercept is not initialized
intercept is not initialized
intercept is not initialized
intercept is not initialized


/Applications/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Applications/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Applications/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Applications/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


ValueError: Floating-point under-/overflow occurred at epoch #1. Scaling input data with StandardScaler or MinMaxScaler might help.